In [1]:
%%capture
!pip install pyspark==3.3.2
!pip install findspark
!pip install pyngrok

In [2]:
import findspark

findspark.init()

from pyspark.sql import SparkSession

spark = SparkSession.builder \
        .appName('homework_5') \
        .getOrCreate()

In [3]:
from pyngrok import ngrok, conf
import getpass

conf.get_default().auth_token = getpass.getpass()

ui_port = 4040
public_url = ngrok.connect(ui_port).public_url
print(f" * ngrok tunnel \"{public_url}\" -> \"http://127.0.0.1:{ui_port}\"")

Enter your authtoken, which can be copied from https://dashboard.ngrok.com/get-started/your-authtoken
··········


 * ngrok tunnel "https://db02-34-73-111-88.ngrok-free.app" -> "http://127.0.0.1:4040"


In [10]:
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/fhv/fhv_tripdata_2019-10.csv.gz
!wget https://github.com/DataTalksClub/nyc-tlc-data/releases/download/misc/taxi_zone_lookup.csv
!gzip -dc fhv_tripdata_2019-10.csv.gz > fhv_tripdata_2019-10.csv

In [5]:
spark.version

'3.3.2'

In [42]:
from pyspark.sql import types
schema_fhv = types.StructType([
    types.StructField('dispatching_base_num', types.StringType(), True),
    types.StructField('pickup_datetime', types.TimestampType(), True),
    types.StructField('dropoff_datetime', types.TimestampType(), True),
    types.StructField('PULocationID', types.IntegerType(), True),
    types.StructField('DOLocationID', types.IntegerType(), True),
    types.StructField('SR_Flag', types.StringType(), True),
    types.StructField('Affiliated_base_number', types.StringType(), True),
])

In [43]:
df_fhv = spark.read \
    .option("header", "true") \
    .schema(schema_fhv) \
    .csv('/content/fhv_tripdata_2019-10.csv')
df_fhv = df_fhv.repartition(6)
df_fhv.write.parquet('fhv/2019/10/')
df_fhv.registerTempTable('fhv_tripdata')

/usr/local/lib/python3.10/dist-packages/pyspark/sql/dataframe.py:229: FutureWarning: Deprecated in 2.0, use createOrReplaceTempView instead.
  warnings.warn("Deprecated in 2.0, use createOrReplaceTempView instead.", FutureWarning)


In [44]:
zones_schema = types.StructType([
    types.StructField('LocationID', types.IntegerType(), True),
    types.StructField('Borough', types.StringType(), True),
    types.StructField('Zone', types.StringType(), True),
    types.StructField('service_zone', types.StringType(), True),
])

In [45]:
df_zones = spark.read \
    .option("header", "true") \
    .schema(zones_schema) \
    .csv('/content/taxi_zone_lookup.csv')
df_zones.registerTempTable('zones_data')

In [47]:
from pyspark.sql import functions as F

df_fhv.where(F.to_date('pickup_datetime')=='2019-10-15').count()
df_fhv.withColumn('diff_hours',(F.col("dropoff_datetime").cast("long") - F.col('pickup_datetime').cast("long"))/F.lit(3600)).agg(F.max('diff_hours')).show()

+---------------+
|max(diff_hours)|
+---------------+
|       631152.5|
+---------------+



In [49]:
spark.sql("""
SELECT
    Zone,
    count(*) AS trips
FROM
    fhv_tripdata
LEFT JOIN
    zones_data ON PULocationID = LocationID
GROUP BY
    Zone
ORDER BY trips
""").show()

+--------------------+-----+
|                Zone|trips|
+--------------------+-----+
|         Jamaica Bay|    1|
|Governor's Island...|    2|
| Green-Wood Cemetery|    5|
|       Broad Channel|    8|
|     Highbridge Park|   14|
|        Battery Park|   15|
|Saint Michaels Ce...|   23|
|Breezy Point/Fort...|   25|
|Marine Park/Floyd...|   26|
|        Astoria Park|   29|
|    Inwood Hill Park|   39|
|       Willets Point|   47|
|Forest Park/Highl...|   53|
|  Brooklyn Navy Yard|   57|
|        Crotona Park|   62|
|        Country Club|   77|
|     Freshkills Park|   89|
|       Prospect Park|   98|
|     Columbia Street|  105|
|  South Williamsburg|  110|
+--------------------+-----+
only showing top 20 rows

